# Creating a Keras Callback that sends notifications on Whatsapp

In [0]:
!pip install twilio

In [0]:
from tensorflow.keras.layers import Dense
from sklearn.datasets import make_classification
#from twilio.rest import Client
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split

In [2]:
def send_message(text):
    account_sid = 'account_ID' #Can be obtained from Twilio Console
    auth_token = 'auth_token'  #Can be obtained from Twilio Console
    client = Client(account_sid, auth_token)

    message = client.messages \
    .create(
         from_='whatsapp:+14155238886',
         body=text,
         to='whatsapp:+918*********'
     )

In [0]:
'''
Creating a custom callback in keras. For more info, see https://www.tensorflow.org/guide/keras/custom_callback and 
    https://keunwoochoi.wordpress.com/2016/07/16/keras-callbacks/
'''

class WhatsappCallBack(Callback): 
  def on_train_begin(self, logs = None):
    self.losses = [] #Initializing the list of losses
    send_message("The training has started") #Sending message on whatsapp at the beginning of the training

  def on_epoch_begin(self, epoch, logs = None):
    pass
    

  def on_epoch_end(self, epoch, logs = None):
    self.losses.append(logs['loss'])

  def on_train_end(self, logs = None):
    send_message("The training has ended") #Sending message on whatsapp at the end of the training
    send_message(self.losses)


In [0]:
cb = WhatsappCallBack() #Creating the Callback

# Creating a Callback that changes the learning rate accoeding to the epoch number

In [0]:
class ChangeLRwithEpoch(Callback):
  def __init__(self, factor=0.5):
    super(ChangeLRwithEpoch, self).__init__()
    self.factor = factor

  def on_train_begin(self, logs = None):
    self.losses = []

  def on_epoch_begin(self, epoch, logs = None):
    epoch+=1

    if(epoch%2 == 0):
      #Calculate the new LR
      self.new_lr = K.get_value(self.model.optimizer.lr)*self.factor

      #Set the new LR
      K.set_value(self.model.optimizer.lr, self.new_lr)

      #Display a print message to reflect the change
      print("The new learning rate is: "+ str(self.new_lr))

In [0]:
cb2 = ChangeLRwithEpoch()

## Creating a sample dataset

In [0]:
x,y = make_classification(n_samples=1000, n_classes=2, n_features=20)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3)

In [0]:
def create_model():
  model = Sequential()
  model.add(Dense(60, input_dim=20, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

In [0]:
model = create_model()


In [36]:
model.fit(x_train,y_train,batch_size=32, epochs=20, callbacks=[cb2], verbose=1) #Fitting the model

Epoch 1/4
22/22 [==============================] - 0s 2ms/step - loss: 0.1398 - accuracy: 0.9686
The new learning rate is: 7.812500371073838e-06
Epoch 2/4
22/22 [==============================] - 0s 2ms/step - loss: 0.1396 - accuracy: 0.9686
Epoch 3/4
22/22 [==============================] - 0s 2ms/step - loss: 0.1399 - accuracy: 0.9686
The new learning rate is: 3.906250185536919e-06
Epoch 4/4
22/22 [==============================] - 0s 2ms/step - loss: 0.1398 - accuracy: 0.9686


In [0]:
model.evaluate(x_test, y_test)